In [ ]:
from openai import OpenAI
from google.adk.tools.base_tool import BaseTool
from google.adk.agents.llm_agent import Agent
import os
from dotenv import load_dotenv
from dataclasses import dataclass
from typing import List, Dict
from openai import InternalServerError

BASE_URL = "https://api.deepgen.app/v1"
MODEL = "gpt-4o"
load_dotenv()


class DeepGenTool(BaseTool):

    def __init__(self, model = MODEL):
        API_KEY = os.getenv("DEEPGEN_API_KEY")
        if not API_KEY:
            raise ValueError("DEEPGEN_API_KEY environment variable not set.")
        
        self.client = OpenAI(api_key=API_KEY, base_url=BASE_URL)
        self.model = model
    
    def chat(self, messages, model=None):
        model = model or self.model
        try:
            response = self.client.chat.completions.create(
                model=model,
                messages= messages
            )
            return response.choices[0].message.content
        except InternalServerError as e:
            return f"Error: {str(e)}"
        

@dataclass
class Debator:
    name:str
    role:str
    tool: DeepGenTool

    def reply(self, debate_history: List[Dict[str, str]], topic : str) -> str:
        messages = []
        sys_message = f"You are {self.name}, a {self.role}. "
        messages.append({"role": "system", "content":sys_message})
    
        for entry in debate_history:
            messages.append({
                "role": "user" if entry["speaker"]!=self.name else "assistant",
                "content" : entry["message"]
            })
        
        if not debate_history:
            messages.append({
                "role": "user",
                "content": f"Please start the debate."
            })
        
        else:
            messages.append({
                "role": "user",
                "content": "Answer the last statement made."
            })
        
        return self.tool.chat(messages)

@dataclass
class Moderator:
    name:str
    tool: DeepGenTool

    def reply(self,p1 : Debator, p2 : Debator,topic: str) -> str:
        messages = []
        sys_message = f"You are {self.name}, the moderator of the debate. "
        messages.append({"role": "system", "content":sys_message})
    
        messages.append({
            "role": "user",
            "content": f"Please introduce the debate topic and the participants."
            f"participant 1 : {p1.name}"
            f"participant 2 : {p2.name}"
        })
        
        return self.tool.chat(messages)

    def conclude(self, debate_history: List[Dict[str, str]],topic : str) -> str:
        messages = []
        sys_message = f"You are {self.name}, the moderator of the debate. "
        messages.append({"role": "system", "content":sys_message})
    
        for entry in debate_history:
            messages.append({
                "role": "user" if entry["speaker"]!=self.name else "assistant",
                "content" : entry["message"]
            })
        
        messages.append({
            "role": "user",
            "content": "Please provide a conclusion to the debate."
        })
        
        return self.tool.chat(messages)


class DebateEngine:
    def __init__(self, participant1, participant2, moderator = None, rounds = 2):
        self.participant1 = participant1
        self.participant2 = participant2
        self.moderator = moderator
        self.rounds = rounds
    
    def run_debate(self,topic):
        debate_history = []

        if self.moderator:
            intro = self.moderator.reply(self.participant1, self.participant2,topic)
            debate_history.append({"speaker": self.moderator.name, "message": intro})

        for round in range(self.rounds):
            p1_reply = self.participant1.reply(debate_history,topic)
            debate_history.append({"speaker": self.participant1.name, "message": p1_reply})

            p2_reply = self.participant2.reply(debate_history,topic)
            debate_history.append({"speaker": self.participant2.name, "message": p2_reply})
        
        if self.moderator:
            conclusion = self.moderator.conclude(debate_history,topic)
            debate_history.append({"speaker": self.moderator.name, "message": conclusion})
        
        return debate_history
    

if __name__ == "__main__":
    tool = DeepGenTool()

    participant1 = Debator(name="Alice", role="Debator", tool=tool)
    participant2 = Debator(name="Bob", role="Debator", tool=tool)
    moderator = Moderator(name="Eve", tool=tool)

    debate_engine = DebateEngine(participant1, participant2, moderator, rounds=3)
    debate_history = debate_engine.run_debate(topic="The impact of AI on society")

    for entry in debate_history:
        print(f"{entry['speaker']}: {entry['message']}\n")


AttributeError: 'Stream' object has no attribute 'choices'